## 데이터 평활
데이터 평활의 이유 : 측정의 오류, 높게 튀는 측정치(정확한 것일지라도 근복적인 공정을 반영한 결과가 아닐 가능성 있음) 두 개를 모두 제거하기 위해 이동평균을 사용해 볼 수 있다.

### 지수평활
최근에 측정된 데이터에 더 큰 가중치를 주고 싶은 경우<br>
시간 $t$에서 평활된 값 : $S_t = d \times S_{t-1} + (1-d) \times x_t$<br>
시간 $t-1$에서 평활된 값 : $S_{t-1} = d \times S_{t-2} + (1-d) \times x_{t-1}$

$\Rightarrow S_t = d\times(d\times S_{t-2} + (1-d)\times x_{t-1})+(1-d)\times x_t$<br>
$\Rightarrow d^3 \times x_{t-3} + d^2 \times x_{t-2} + d \times x_{t-1}$과 같은 연쇄적 형태

In [6]:
import pandas as pd
air = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/AirPassengers.csv", names=['Date', 'Passengers'])

# ewm 함수는 다양한 감쇠요인을 적용하여 평활할 수 있다. alpha 값이 클수록 값의 갱신은 있는 그대로의 현재 값에 가깝게 더 빨리 갱신됨
air['Smooth.5'] = air.ewm(alpha = 0.5).mean().Passengers
air['Smooth.1'] = air.ewm(alpha = 0.9).mean().Passengers
print(air)

        Date  Passengers    Smooth.5    Smooth.1
0    1949-01         112  112.000000  112.000000
1    1949-02         118  116.000000  117.454545
2    1949-03         132  125.142857  130.558559
3    1949-04         129  127.200000  129.155716
4    1949-05         121  124.000000  121.815498
..       ...         ...         ...         ...
139  1960-08         606  582.096411  606.665454
140  1960-09         508  545.048205  517.866545
141  1960-10         461  503.024103  466.686655
142  1960-11         390  446.512051  397.668665
143  1960-12         432  439.256026  428.566867

[144 rows x 4 columns]


# 시간대
복잡성
- 시간대는 정치적, 사회적 결정에 의해 형성된다.
- 언어 간 또는 HTTP 프로토콜을 통해 표준 시간대 정보를 전송하는 표준 방법이 없다.
- 시간대에 이름을 짓거나 일광 절약(daylight saving)의 시작과 종료일을 결정하기 위한 단일화된 프로토콜이 존재하지 않는다.
- 일광 적약으로 한 해에 몇 시간씩 시간의 중복이 발생한다.

파이썬의 datetime.datetime.now()는 시간대를 인식한 타임스탬를 반환하지 않는다. 또 시간에 관련한 객체를 pickle로 저장할 시 문제가 생길 수 있다.

In [7]:
# 라이브러리
import datetime, pytz, dateutil
# pandas는 pytz 및 dateutil 라이브러리에 기반한 시간대에 편리한 기능을 제공

In [9]:
import numpy as np
import pandas as pd
from IPython.display import display

In [10]:
display(datetime.datetime.utcnow())
# 시간대를 부여해주지는 않았지만 자신의 시간대에 맞춰 표시된다.
display(datetime.datetime.now())
# tz는 timezone object
display(datetime.datetime.now(tz = datetime.timezone.utc))

datetime.datetime(2022, 2, 20, 6, 13, 19, 537611)

datetime.datetime(2022, 2, 20, 15, 13, 19, 541599)

datetime.datetime(2022, 2, 20, 6, 13, 19, 544982, tzinfo=datetime.timezone.utc)

In [11]:
# timezone 객체를 생성할 수 있다. 예시는 태평양 표준시
western = pytz.timezone('US/Pacific')
western.zone

'US/Pacific'

In [12]:
# timezone 객체를 이용해 시간대를 현지화할 수 있다.
loc_dt = western.localize(datetime.datetime.now())
display(loc_dt)

datetime.datetime(2022, 2, 20, 15, 15, 34, 649043, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

In [13]:
london_tz = pytz.timezone('Europe/London')
london_dt = loc_dt.astimezone(london_tz)
london_dt

datetime.datetime(2022, 2, 20, 23, 15, 34, 649043, tzinfo=<DstTzInfo 'Europe/London' GMT0:00:00 STD>)

In [14]:
event1 = datetime.datetime.now(london_tz)
event2 = datetime.datetime.now(western)
display(event1)
display(event2)

datetime.datetime(2022, 2, 20, 6, 16, 5, 550346, tzinfo=<DstTzInfo 'Europe/London' GMT0:00:00 STD>)

datetime.datetime(2022, 2, 19, 22, 16, 5, 550346, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

In [15]:
# 서로의 시간대가 적절하게 레이블링되지 않았기 때문에 잘못된 시간간격을 계산한다.
event2 - event1

datetime.timedelta(0)

In [16]:
event1 = london_tz.localize(datetime.datetime.now()).astimezone(datetime.timezone.utc)
event2 = western.localize(datetime.datetime.now()).astimezone(datetime.timezone.utc)

In [17]:
display(event1)
display(event2)

datetime.datetime(2022, 2, 20, 15, 16, 5, 968092, tzinfo=datetime.timezone.utc)

datetime.datetime(2022, 2, 20, 23, 16, 5, 969089, tzinfo=datetime.timezone.utc)

In [18]:
event2 - event1

datetime.timedelta(seconds=28800, microseconds=997)

In [19]:
# 일반 시간대를 확인할 수 있다.
print(pytz.common_timezones[:10])

['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa', 'Africa/Algiers', 'Africa/Asmara', 'Africa/Bamako', 'Africa/Bangui', 'Africa/Banjul', 'Africa/Bissau', 'Africa/Blantyre']


In [20]:
# 국가별 시간대를 확인할 수 있다.
print(pytz.country_timezones('RU'))

['Europe/Kaliningrad', 'Europe/Moscow', 'Europe/Kirov', 'Europe/Astrakhan', 'Europe/Volgograd', 'Europe/Saratov', 'Europe/Ulyanovsk', 'Europe/Samara', 'Asia/Yekaterinburg', 'Asia/Omsk', 'Asia/Novosibirsk', 'Asia/Barnaul', 'Asia/Tomsk', 'Asia/Novokuznetsk', 'Asia/Krasnoyarsk', 'Asia/Irkutsk', 'Asia/Chita', 'Asia/Yakutsk', 'Asia/Khandyga', 'Asia/Vladivostok', 'Asia/Ust-Nera', 'Asia/Magadan', 'Asia/Sakhalin', 'Asia/Srednekolymsk', 'Asia/Kamchatka', 'Asia/Anadyr']
